<a href="https://colab.research.google.com/github/sfnesbit/Wildfire-Risk-Assessment/blob/main/FireVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Visualize burn map from Google Earth Engine**


### **Install Earth Engine API**

In [ ]:
!pip install earthengine-api --upgrade

### **Install Folium**

In [ ]:
!pip install folium

### **Prevent re-import errors**

In [ ]:
%load_ext autoreload
%autoreload 2

### **Imports**

In [ ]:
from IPython.core.display import display, HTML
import numpy as np
import folium
import os
import ee

### **Sign-in to Google Earth Engine**

In [ ]:
ee.Authenticate()
try:
  ee.Initialize()
  print("Earth Engine initilized successfully!")
except ee.EEException as e:
  print("Earth Engine could not be initialized!")

Earth Engine initilized successfully!


## **Build Fire Map Functions**

### **Adds an Earth Engine layer to a Folium map**

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

### **Builds the Folium map given an <*ee.Image*> object**

In [ ]:
def buildFireMap(image, bounds=None, burn_color='FF00FF', tile_color='white', band='T21'):
  image_viz_params = {'min': 0.5, 'max': 1, 'palette': [burn_color, burn_color]}

  white_tile = {'tile': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
                'attr':'Tiles &copy; Esri &mdash; Source: USGS, Esri, TANA, DeLorme, and NPS'}
  colored_tile = {'tile':'https://stamen-tiles-{s}.a.ssl.fastly.net/terrain-background/{z}/{x}/{y}{r}.png',
                'attr': 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'}
  tile = white_tile if tile_color == 'white' else colored_tile
  
  # Build map from selected tile type
  fMap = folium.Map(location=[37.5010, -122.1899], zoom_start=10,
                      tiles = tile['tile'],
                      attr = tile['attr'])
  
  # Bound image to [[sw_long, sw_lat], [ne_long, ne_lat]]
  if bounds is not None:
    fMap.fit_bounds(bounds)

  # Add burn layer to image
  fMap.add_ee_layer(image, image_viz_params, band)
  return fMap

### **Easy-use function to get burn map across a date range and bounding box**

In [ ]:
# start_date: date to start fire data collection from
#   'YYYY-MM-DD'
# end_date: final date of fire data collection
#   'YYYY-MM-DD'
# southwest_bound: lower left corner of bounding box to fit map to
#    [lat, long]
# northeast_bound: upper right corner of bounding box to fit map to
#    [lat, long]
# map_tile: what terrain type you want to the map to look like
#    'white': a grayscale relief map
#    'color': a colored relief map
# burn_color: what color do you want the burned areas to be:
#    '#FF00FF': default magenta
def getBurnMap(start_date, end_date, southwest_bound, northeast_bound, map_tile='white', burn_color='#FF3617'):
  dataset = ee.ImageCollection('FIRMS').filter(
        ee.Filter.date(start_date, end_date))
  fires = dataset.select('T21') 
  image = fires.sum()

  region = ee.Geometry.BBox(southwest_bound[1],southwest_bound[0], northeast_bound[1],northeast_bound[0])
  meanDictionary = image.reduceRegion(
    reducer= ee.Reducer.mean(),
    geometry= region,
    scale= 30,
    maxPixels= 1e9)
  
  fireMap = buildFireMap(image, [southwest_bound, northeast_bound], burn_color=burn_color, tile_color=map_tile)
  return fireMap, meanDictionary

### **Miscellaneous Helper Functions**

In [ ]:
# Make a bounding box around a [lat,long] pair with dimensions of 2radius x 2radius
def buildCoords(coords, radius):
  southwest = [coords[0]-radius, coords[1]-radius]
  northeast = [coords[0]+radius, coords[1]+radius]
  return southwest, northeast
  
def sideBySide(map1, map2):
  return HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 40%; margin: 0 auto; border: 2px solid black"></iframe>'
              '<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 40%; margin: 0 auto; border: 2px solid black"></iframe>'
              .format(map1.get_root().render().replace('"', '&quot;'),500,500,
                      map2.get_root().render().replace('"', '&quot;'),500,500))

### **Determine if a burn dictionary contains details of a fire in the original bounded region**

In [ ]:
# Given a the reduced dictionary of the area, determine if the pixel 
# count is higher than a given tolerance
def didAreaBurn(burnDictionary, band='T21', tolerance=0):
  burnAmount = burnDictionary.getInfo()[band] 
  burnAmount = 0 if burnAmount is None else burnAmount
  return burnAmount > tolerance

## **Create burn maps for San Francisco and Santa Cruz**

### October 20th 2020 CZU Lightning Complex Fires

In [ ]:
radius = 0.1
SF_sw, SF_nw = buildCoords([37.75, -122.40], radius)
SC_sw, SC_nw = buildCoords([37.15, -122.20], radius)

SFBurnMap, SFBurnDictionary = getBurnMap('2020-08-15', '2020-08-20', SF_sw, SF_nw, 'color')
SCBurnMap, SCBurnDictionary = getBurnMap('2020-08-15', '2020-08-20', SC_sw, SC_nw, 'color')

### **Determine whether the areas burned and display their fire maps**

In [ ]:
print(f'San Francisco: Burned? {didAreaBurn(SFBurnDictionary)}')
print(f'Santa Cruz: Burned? {didAreaBurn(SCBurnDictionary)}')
sideBySide(SFBurnMap, SCBurnMap)

San Francisco: Burned? False
Santa Cruz: Burned? True


C:\Users\sfnes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### **Save outputs**

In [ ]:
SCBurnMap.save('../outputs/CZUFireBurnMap.html')